# Mount One Drive

In [1]:
# mount google drive to colab
from google.colab import drive
import os

n = 140
drive.mount('/content/drive')
print("#" * n)

# change working directory
print("### colabs default working directory")

print("###\n### path: {}".format(os.getcwd()))
print("#" * n)
print("### setted working directory")

%cd drive/Shareddrives/MADS_Milestone2/


print("###\n### {}".format(os.getcwd()))
print("#" * n)

Mounted at /content/drive
############################################################################################################################################
### colabs default working directory
###
### path: /content
############################################################################################################################################
### setted working directory
/content/drive/Shareddrives/MADS_Milestone2
###
### /content/drive/Shareddrives/MADS_Milestone2
############################################################################################################################################


# Create a network of biological functions




Considerations/problems/questions?
- should we only consider biological functions which are associated with at least one drug target or one disease protein, either directly or implicitly through their children to focus on the biological functions most relevant to treatment and disease?

## Download basic Gene Ontologies 

# [go-basic.obo](http://geneontology.org/docs/download-ontology/) is the basic version of the Gene Ontology, filtered such that the graph is guaranteed to be acyclic and annotations can be propagated up the graph. The relations included are **is a**, **part of**, **regulates**, **negatively regulates** and **positively regulates**

In [ ]:
# download dataset
# !wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ http://purl.obolibrary.org/obo/go/go-basic.obo

--2021-10-27 20:05:11--  http://purl.obolibrary.org/obo/go/go-basic.obo
Resolving purl.obolibrary.org (purl.obolibrary.org)... 52.3.123.63
Connecting to purl.obolibrary.org (purl.obolibrary.org)|52.3.123.63|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://current.geneontology.org/ontology/go-basic.obo [following]
--2021-10-27 20:05:12--  http://current.geneontology.org/ontology/go-basic.obo
Resolving current.geneontology.org (current.geneontology.org)... 13.224.54.37, 13.224.54.113, 13.224.54.196, ...
Connecting to current.geneontology.org (current.geneontology.org)|13.224.54.37|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30901321 (29M) [text/obo]
Saving to: ‘/content/drive/Shareddrives/MADS_Milestone2/dat/go-basic.obo.1’

go-basic.obo.1      100%[===================>]  29.47M  46.4MB/s    in 0.6s    

2021-10-27 20:05:12 (46.4 MB/s) - ‘/content/drive/Shareddrives/MADS_Milestone2/dat/go-basic.obo.1’ saved [30901321/309013

## import libraries

In [2]:
try:
    import obonet
    print("obnet was alredy installed and is imported now")   

except ModuleNotFoundError as err:
    !pip install obonet
    import obonet
    print("obnet was not imported before")   
    
    # Error handling
    print(err)    

import networkx as nx 
import pandas as pd
import numpy as np

obnet was not imported before
No module named 'obonet'


In [3]:
# DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
# df = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv")#.drop(columns="Unnamed: 0")

## load biological function into netwroknx graph

In [ ]:
GO = obonet.read_obo("dat/go-basic.obo")

In [ ]:
# Number of nodes
print("Number of nodes: {}".format(len(GO)))
# Number of edges
print("Number of edges: {}".format(GO.number_of_edges()))
# Check if the ontology is a directed acyclic graph (DAG)
print("Is it really a directed acyclic graph: {}".format({True: "Yes", False: "No"}[nx.is_directed_acyclic_graph(GO)]))

Number of nodes: 43850
Number of edges: 86697
Is it really a directed acyclic graph: Yes


In [ ]:
# pd.DataFrame([list( GO.nodes(data=True))[0][1]])

convert nodes together with the attributes to a pandas dataframe

In [ ]:
# the commented code below is breaking colab due to resource limitations (too much RAM) therefore we iteratre through nodes
# pd.DataFrame(nx.to_dict_of_dicts(GO))

bf = pd.DataFrame()

for i in list( GO.nodes(data=True)):
  
  new = pd.DataFrame([i[1]], index=[i[0]])
  new_col = list(set(list(new.columns)) - set(bf.columns))

  bf[new_col] = None
  bf = pd.concat([bf,new])


KeyboardInterrupt: ignored

In [ ]:
# export dataframe as intermediate network between biological function nodes
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
bf.to_csv(DIR + "res/intermediate_biological_function.csv")

# map the biological function to the proteins 

In [ ]:
# import mygene library to connect to the API
try:
    import mygene
except ModuleNotFoundError as err:
    !pip install mygene
    import mygene
    # Error handling
    print(err)

# project directory
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
# load the biological function dataframe
bf2 = pd.read_csv(DIR + "res/intermediate_biological_function.csv",
                  converters={"is_a": lambda x: x.replace("'", '').strip("'[]'").split(", "),
                              "relationship": lambda x: x.replace("'", '').strip("'[]'").split(", ")}).rename(columns={"Unnamed: 0": "GO"})

# lets concentrate only on biological process - cell location is unimportant for us to know
bf2 = bf2[bf2.namespace=="biological_process"]

# make API connection
mg = mygene.MyGeneInfo()

# fetch all genes for each biological function
out = mg.querymany(bf2.GO.tolist(), scopes='GO', fields='entrezgene,symbol', species='human', returnall=True)



querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-28503...done.
Finished.
9114 input query terms found dup hits:
	[('GO:0000002', 10), ('GO:0000003', 4), ('GO:0000012', 10), ('GO:0000017', 2), ('GO:0000018', 6), ('
16024 input query terms found 

convert the fetched proteins for each biological function to an intermediate pandas data frame 

In [ ]:
bp = pd.DataFrame()

for i in out["out"]:

  new = pd.DataFrame([i])
  new_col = list(set(list(new.columns)) - set(bp.columns))

  bp[new_col] = None
  bp = pd.concat([bp,new])

bp.reset_index(drop=True).to_csv(DIR + "res/intermediate_protein_biological_function_map.csv")

bp

,query,notfound,_score,entrezgene,_id,symbol,go
0,GO:0000001,True,NaN,None,None,None,None
0,GO:0000002,NaN,16.971151,55186,55186,SLC25A36,"{'BP': [{'evidence': 'IDA', 'id': 'GO:0000002'..."
0,GO:0000002,NaN,16.971151,9361,9361,LONP1,"{'BP': [{'evidence': 'NAS', 'id': 'GO:0000002'..."
0,GO:0000002,NaN,16.971151,291,291,SLC25A4,"{'BP': [{'evidence': 'TAS', 'id': 'GO:0000002'..."
0,GO:0000002,NaN,16.971151,4976,4976,OPA1,"{'BP': [{'evidence': 'IMP', 'id': 'GO:0000002'..."
...,...,...,...,...,...,...,...
0,GO:2001313,True,NaN,NaN,NaN,NaN,NaN
0,GO:2001314,True,NaN,NaN,NaN,NaN,NaN
0,GO:2001315,True,NaN,NaN,NaN,NaN,NaN
0,GO:2001316,True,NaN,NaN,NaN,NaN,NaN


In [ ]:

# import 
bf_map = pd.read_csv(DIR + "res/intermediate_protein_biological_function_map.csv").drop(columns="Unnamed: 0").fillna(False)

# bring the mapped proteins in a format such that we can use explode to get all the edges in a pandas dataframe
bf_map_cond = bf_map.groupby("query").agg({"notfound": lambda x: list(map(bool, x)),
                                    "_id": lambda x: list(map(int, x)),
                                   "entrezgene": lambda x: list(map(int, x)),
                                   "symbol": lambda x: list(map(str, x))})

# merge the edges within biological functions and the biological functions with the proteins 
bf = bf2.merge(bf_map_cond, left_on="GO", right_on="query", how="left")
 

# convert NaN to list of NaN so we can explode it 
col =  ["is_a",	"synonym", "xref", "subset", "alt_id", "relationship", "comment"]
for c in col:
  if c == "relationship" or c == "is_a":
    ind = [True if i == [''] else False for i in bf[c].tolist()]
    bf[c][ind] = [[np.nan]]
  
  bf.loc[bf.loc[:,c].isnull(), c] = [[np.nan]]

bf



,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
0,GO:0000001,mitochondrion inheritance,biological_process,"[GO:0048308, GO:0048311]","['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]
1,GO:0000002,mitochondrial genome maintenance,biological_process,[GO:0007005],[nan],"""The maintenance of the structure and integrit...",[nan],[nan],[nan],[nan],[nan],"[False, False, False, False, False, False, Fal...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[PIF1, SLC25A33, MGME1, TYMP, SLC25A36, MPV17,..."
2,GO:0000003,reproduction,biological_process,[GO:0008150],"['""reproductive physiological process"" EXACT []']","""The production of new individuals that contai...",['Wikipedia:Reproduction'],"['goslim_agr', 'goslim_chembl', 'goslim_flybas...","['GO:0019952', 'GO:0050876']",[nan],[nan],"[False, False, False, False]","[286826, 2797, 8510, 2796]","[286826, 2797, 8510, 2796]","[LIN9, GNRH2, MMP23B, GNRH1]"
3,GO:0000011,vacuole inheritance,biological_process,"[GO:0007033, GO:0048308]",[nan],"""The distribution of vacuoles into daughter ce...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]
4,GO:0000012,single strand break repair,biological_process,[GO:0006281],[nan],"""The repair of single strand breaks in DNA. Re...",[nan],[nan],[nan],[nan],[nan],"[False, False, False, False, False, False, Fal...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[TDP1, XNDC1N, ERCC8, ERCC6, APLF, APTX, XRCC1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28498,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"[GO:0006040, GO:0006793, GO:0009225]","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways involving...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]
28499,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"[GO:0009227, GO:0046348, GO:2001313]","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]
28500,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"[GO:0009226, GO:0046349, GO:2001313]","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]
28501,GO:2001316,kojic acid metabolic process,biological_process,"[GO:0034308, GO:0042180, GO:0046483, GO:012025...","['""5-hydroxy-2-(hydroxymethyl)-4H-pyran-4-one ...","""The chemical reactions and pathways involving...",[nan],[nan],[nan],[nan],[nan],[True],[0],[0],[False]


In [ ]:
# explode the lists in pd dataframe
bf = (bf.set_index('GO')
   .apply(lambda x: x.apply(pd.Series).stack())
   .reset_index()
   .drop('level_1', 1))
bf

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
0,GO:0000001,mitochondrion inheritance,biological_process,GO:0048308,"['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False
1,GO:0000001,NaN,NaN,GO:0048311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000002,mitochondrial genome maintenance,biological_process,GO:0007005,NaN,"""The maintenance of the structure and integrit...",NaN,NaN,NaN,NaN,NaN,False,80119.0,80119.0,PIF1
3,GO:0000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,84275.0,84275.0,SLC25A33
4,GO:0000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,92667.0,92667.0,MGME1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90747,GO:2001317,NaN,NaN,GO:0034309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90748,GO:2001317,NaN,NaN,GO:0042181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90749,GO:2001317,NaN,NaN,GO:0120255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90750,GO:2001317,NaN,NaN,GO:1901362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## select the different edge types of the biological functions and merge it

In [ ]:
# construct a lookup table to map the biological function keys to the names
GO_conv = bf.drop_duplicates("GO").set_index("GO").loc[:,["name"]].to_dict().copy()

# since we exploded the rows we need to propagate the information for name and namespace since they were not lists
bf.name = bf.name.fillna(method="ffill")
bf.namespace = bf.namespace.fillna(method="ffill")

# structure the dataframe for all edges within biological function of edge type is_a
bf1 = bf[bf.is_a.notnull()].loc[:, ["GO", "name", "namespace", "is_a"]].rename(columns={"GO": "node1", "name": "node1_name", "namespace": "node1_type", "is_a": "node2"}).copy()
bf1["edge_type"] = "is_a"
bf1["node2_name"] = bf1.node2
bf1[["node2_name"]] = bf1[["node2_name"]].replace(GO_conv["name"])
bf1["node2_type"] = bf1["node1_type"]

# structure all edges for proteins and biological functions
bf2 = bf[((bf.entrezgene.notnull()) & (bf.entrezgene != 0))].loc[:, ["GO", "name", "namespace", "entrezgene", "symbol"]].rename(columns={"GO": "node1", "name": "node1_name", "namespace": "node1_type", "entrezgene": "node2", "symbol": "node2_name"}).copy()
bf2["node2_type"] = "protein"
bf2["edge_type"] = "indirect"

# structure all edges for all other edge types such as regulates part of negative or postive reguklarion within biological function
bf3 = bf[bf.edge_type.notnull()].loc[:, ["GO", "name", "namespace", "node2", "edge_type"]].rename(columns={"GO": "node1", "name": "node1_name", "namespace": "node1_type"}).copy()
bf3["node2_name"] = bf3.node2
bf3[["node2_name"]].replace(GO_conv["name"])
bf3["node2_name"] = bf3.node2
bf3[["node2_name"]] = bf3[["node2_name"]].replace(GO_conv["name"])
bf3["node2_type"] = bf3["node1_type"]

bf3

,node1,node1_name,node1_type,node2,edge_type,node2_name
30,GO:0000018,regulation of DNA recombination,biological_process,GO:0006310,regulates,DNA recombination
36,GO:0000019,regulation of mitotic recombination,biological_process,GO:0006312,regulates,mitotic recombination
38,GO:0000022,mitotic spindle elongation,biological_process,GO:0000070,part_of,mitotic sister chromatid segregation
39,GO:0000022,mitotic spindle elongation,biological_process,GO:0007052,part_of,mitotic spindle organization
46,GO:0000027,ribosomal large subunit assembly,biological_process,GO:0042255,part_of,ribosome assembly
...,...,...,...,...,...,...
90644,GO:2001284,regulation of BMP secretion,biological_process,GO:0038055,regulates,BMP secretion
90647,GO:2001285,negative regulation of BMP secretion,biological_process,GO:0038055,negatively_regulates,BMP secretion
90651,GO:2001286,regulation of caveolin-mediated endocytosis,biological_process,GO:0072584,regulates,caveolin-mediated endocytosis
90652,GO:2001287,negative regulation of caveolin-mediated endoc...,biological_process,GO:0072584,negatively_regulates,caveolin-mediated endocytosis


merge all the edges and export it as final biological function network

In [ ]:
BF = bf1.append(bf2).append(bf3)
BF
BF.to_csv(DIR + "res/final_bf_bf_and_bf_protein_interaction_network.csv")

NameError: ignored

In [ ]:
bf1

,node1,node1_name,node1_type,node2,edge_type,node2_name,node2_type
0,GO:0000001,mitochondrion inheritance,biological_process,GO:0048308,is_a,organelle inheritance,biological_process
1,GO:0000001,mitochondrion inheritance,biological_process,GO:0048311,is_a,mitochondrion distribution,biological_process
2,GO:0000002,mitochondrial genome maintenance,biological_process,GO:0007005,is_a,mitochondrion organization,biological_process
12,GO:0000003,reproduction,biological_process,GO:0008150,is_a,biological_process,biological_process
16,GO:0000011,vacuole inheritance,biological_process,GO:0007033,is_a,vacuole organization,biological_process
...,...,...,...,...,...,...,...
90747,GO:2001317,kojic acid biosynthetic process,biological_process,GO:0034309,is_a,primary alcohol biosynthetic process,biological_process
90748,GO:2001317,kojic acid biosynthetic process,biological_process,GO:0042181,is_a,ketone biosynthetic process,biological_process
90749,GO:2001317,kojic acid biosynthetic process,biological_process,GO:0120255,is_a,olefinic compound biosynthetic process,biological_process
90750,GO:2001317,kojic acid biosynthetic process,biological_process,GO:1901362,is_a,organic cyclic compound biosynthetic process,biological_process


In [4]:
bf =  pd.read_csv(DIR + "res/final_bf_bf_and_bf_protein_interaction_network.csv")

In [7]:
bf.apply(lambda x: (x["node1_type"] == "biological_process") and (x["node2_type"] == "biological_process"), axis=1).sum() 

66277

In [8]:
bf.apply(lambda x: (x["node1_type"] == "biological_process") and (x["node2_type"] == "protein"), axis=1).sum()

57386

In [13]:
print(len(set(bf[bf.apply(lambda x: (x["node1_type"] == "biological_process") and (x["node2_type"] == "protein"), axis=1)].loc[:,"node1"])))
len(set(bf[bf.apply(lambda x: (x["node1_type"] == "biological_process") and (x["node2_type"] == "protein"), axis=1)].loc[:,"node2"]))

12488


12771

In [ ]:
GO_conv["name"][bf1.node2_name.iloc[200]]

'RNA splicing'

In [ ]:
# bf1["node2_name"] = bf1.node2
GO_conv = bf.drop_duplicates("GO").set_index("GO").loc[:,["name"]].to_dict().copy()
bf1[["node2_name"]] = bf1[["node2_name"]].replace(GO_conv["name"])


,node2_name
0,organelle inheritance
1,mitochondrion distribution
2,mitochondrion organization
12,biological_process
16,vacuole organization
...,...
90747,primary alcohol biosynthetic process
90748,ketone biosynthetic process
90749,olefinic compound biosynthetic process
90750,organic cyclic compound biosynthetic process


In [ ]:
bf.name = bf.name.fillna(method="ffill")
bf

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol,rel,edge_type,node2
0,GO:0000001,mitochondrion inheritance,biological_process,GO:0048308,"['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False,False,NaN,NaN
1,GO:0000001,mitochondrion inheritance,NaN,GO:0048311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
2,GO:0000002,mitochondrial genome maintenance,biological_process,GO:0007005,NaN,"""The maintenance of the structure and integrit...",NaN,NaN,NaN,NaN,NaN,False,80119.0,80119.0,PIF1,False,NaN,NaN
3,GO:0000002,mitochondrial genome maintenance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,84275.0,84275.0,SLC25A33,False,NaN,NaN
4,GO:0000002,mitochondrial genome maintenance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,92667.0,92667.0,MGME1,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90747,GO:2001317,kojic acid biosynthetic process,NaN,GO:0034309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
90748,GO:2001317,kojic acid biosynthetic process,NaN,GO:0042181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
90749,GO:2001317,kojic acid biosynthetic process,NaN,GO:0120255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
90750,GO:2001317,kojic acid biosynthetic process,NaN,GO:1901362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [ ]:
bf["rel"] = [s.split() if isinstance(s, str) else False for s in bf.relationship ]
bf["edge_type"] =  [i[0] if i != False else np.nan for i in bf.rel]
bf["node2"] =  [i[1] if i != False else np.nan for i in bf.rel]
bf.node2

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
90747    NaN
90748    NaN
90749    NaN
90750    NaN
90751    NaN
Name: node2, Length: 90752, dtype: object

In [ ]:
for i,r in bf.iterrows():
  if isinstance(r["is_a"], str) and r["edge_type"] != False:
    print(r)

Streaming output truncated to the last 5000 lines.
synonym                                                       NaN
def             "Any process that modulates the frequency, rat...
xref                                                          NaN
subset                                                        NaN
alt_id                                                        NaN
relationship                                 regulates GO:0014841
comment                                                       NaN
notfound                                                    False
_id                                                         94104
entrezgene                                                  94104
symbol                                                     PAXBP1
rel                                       [regulates, GO:0014841]
edge_type                                               regulates
Name: 18687, dtype: object
GO                                                     GO:001484

KeyboardInterrupt: ignored

In [ ]:
np.isnan(bf.relationship[0])

True

In [ ]:
bf.relationship.iloc[30]

'regulates GO:0006310'

In [ ]:
bf[bf.relationship.notnull()]

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
30,GO:0000018,regulation of DNA recombination,biological_process,GO:0051052,NaN,"""Any process that modulates the frequency, rat...",NaN,NaN,NaN,regulates GO:0006310,NaN,False,9984.0,9984.0,THOC1
36,GO:0000019,regulation of mitotic recombination,biological_process,GO:0000018,"['""regulation of recombination within rDNA rep...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,regulates GO:0006312,NaN,False,4361.0,4361.0,MRE11
38,GO:0000022,mitotic spindle elongation,biological_process,GO:0051231,"['""microtubule sliding involved in mitotic spi...","""The cell cycle process in which the distance ...",NaN,NaN,['GO:1905121'],part_of GO:0000070,NaN,False,9493.0,9493.0,KIF23
39,GO:0000022,NaN,NaN,GO:1903047,NaN,NaN,NaN,NaN,NaN,part_of GO:0007052,NaN,False,9055.0,9055.0,PRC1
46,GO:0000027,ribosomal large subunit assembly,biological_process,GO:0022618,"['""50S ribosomal subunit assembly"" NARROW [GOC...","""The aggregation, arrangement and bonding toge...",NaN,NaN,NaN,part_of GO:0042255,NaN,False,140801.0,140801.0,RPL10L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90644,GO:2001284,regulation of BMP secretion,biological_process,GO:0010646,"['""regulation of BMP protein secretion"" EXACT ...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,regulates GO:0038055,NaN,True,0.0,0.0,False
90647,GO:2001285,negative regulation of BMP secretion,biological_process,GO:0010648,"['""negative regulation of BMP protein secretio...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,negatively_regulates GO:0038055,NaN,True,0.0,0.0,False
90651,GO:2001286,regulation of caveolin-mediated endocytosis,biological_process,GO:0030100,"['""regulation of caveolae-dependent endocytosi...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,regulates GO:0072584,NaN,False,6714.0,6714.0,SRC
90652,GO:2001287,negative regulation of caveolin-mediated endoc...,biological_process,GO:0045806,"['""negative regulation of caveolae-dependent e...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,negatively_regulates GO:0072584,NaN,False,150696.0,150696.0,PROM2


In [ ]:
bf2.relationship.iloc[30]

"['part_of GO:0051329']"

In [ ]:
col =  ["is_a",	"synonym", "xref", "subset", "alt_id", "relationship"]
for c in col:
  isna = bf2[c].isna()
  bf2.loc[isna, c] = pd.Series([[]] * isna.sum()).values

bf2.is_a.apply(literal_eval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


NameError: ignored

In [ ]:
bf2.loc[:,"is_a"].apply(literal_eval)

ValueError: ignored

In [ ]:
bf2.dtypes

GO              object
name            object
namespace       object
is_a            object
synonym         object
def             object
xref            object
subset          object
alt_id          object
relationship    object
comment         object
dtype: object

In [ ]:
bf = bf2.merge(bf_map_cond, left_on="GO", right_on="query", how="left")
bf

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
0,GO:0000001,mitochondrion inheritance,biological_process,"['GO:0048308', 'GO:0048311']","['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]
1,GO:0000002,mitochondrial genome maintenance,biological_process,['GO:0007005'],NaN,"""The maintenance of the structure and integrit...",NaN,NaN,NaN,NaN,NaN,"[False, False, False, False, False, False, Fal...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[PIF1, SLC25A33, MGME1, TYMP, SLC25A36, MPV17,..."
2,GO:0000003,reproduction,biological_process,['GO:0008150'],"['""reproductive physiological process"" EXACT []']","""The production of new individuals that contai...",['Wikipedia:Reproduction'],"['goslim_agr', 'goslim_chembl', 'goslim_flybas...","['GO:0019952', 'GO:0050876']",NaN,NaN,"[False, False, False, False]","[286826, 2797, 8510, 2796]","[286826, 2797, 8510, 2796]","[LIN9, GNRH2, MMP23B, GNRH1]"
3,GO:0000011,vacuole inheritance,biological_process,"['GO:0007033', 'GO:0048308']",NaN,"""The distribution of vacuoles into daughter ce...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]
4,GO:0000012,single strand break repair,biological_process,['GO:0006281'],NaN,"""The repair of single strand breaks in DNA. Re...",NaN,NaN,NaN,NaN,NaN,"[False, False, False, False, False, False, Fal...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[TDP1, XNDC1N, ERCC8, ERCC6, APLF, APTX, XRCC1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28498,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0006040', 'GO:0006793', 'GO:0009225']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways involving...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]
28499,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009227', 'GO:0046348', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]
28500,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009226', 'GO:0046349', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]
28501,GO:2001316,kojic acid metabolic process,biological_process,"['GO:0034308', 'GO:0042180', 'GO:0046483', 'GO...","['""5-hydroxy-2-(hydroxymethyl)-4H-pyran-4-one ...","""The chemical reactions and pathways involving...",NaN,NaN,NaN,NaN,NaN,[True],[0],[0],[False]


In [ ]:
# bf.loc[:,"is_a"][bf.is_a.isnull()]
bf.is_a[0]

"['GO:0048308', 'GO:0048311']"

In [ ]:
explode(bf, lst_cols=list('_id'))

KeyError: ignored

In [ ]:
col =  ["is_a",	"synonym", "xref", "subset", "alt_id", "relationship"]
for c in col:
  bf.loc[bf.loc[:,c].isnull(), c] = [[]]
bf

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
0,GO:0000001,mitochondrion inheritance,biological_process,"['GO:0048308', 'GO:0048311']","['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",[],[],[],[],NaN,[True],[0],[0],[False]
1,GO:0000002,mitochondrial genome maintenance,biological_process,['GO:0007005'],[],"""The maintenance of the structure and integrit...",[],[],[],[],NaN,"[False, False, False, False, False, False, Fal...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[PIF1, SLC25A33, MGME1, TYMP, SLC25A36, MPV17,..."
2,GO:0000003,reproduction,biological_process,['GO:0008150'],"['""reproductive physiological process"" EXACT []']","""The production of new individuals that contai...",['Wikipedia:Reproduction'],"['goslim_agr', 'goslim_chembl', 'goslim_flybas...","['GO:0019952', 'GO:0050876']",[],NaN,"[False, False, False, False]","[286826, 2797, 8510, 2796]","[286826, 2797, 8510, 2796]","[LIN9, GNRH2, MMP23B, GNRH1]"
3,GO:0000011,vacuole inheritance,biological_process,"['GO:0007033', 'GO:0048308']",[],"""The distribution of vacuoles into daughter ce...",[],[],[],[],NaN,[True],[0],[0],[False]
4,GO:0000012,single strand break repair,biological_process,['GO:0006281'],[],"""The repair of single strand breaks in DNA. Re...",[],[],[],[],NaN,"[False, False, False, False, False, False, Fal...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[TDP1, XNDC1N, ERCC8, ERCC6, APLF, APTX, XRCC1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28498,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0006040', 'GO:0006793', 'GO:0009225']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways involving...",[],[],[],[],NaN,[True],[0],[0],[False]
28499,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009227', 'GO:0046348', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",[],[],[],[],NaN,[True],[0],[0],[False]
28500,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009226', 'GO:0046349', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",[],[],[],[],NaN,[True],[0],[0],[False]
28501,GO:2001316,kojic acid metabolic process,biological_process,"['GO:0034308', 'GO:0042180', 'GO:0046483', 'GO...","['""5-hydroxy-2-(hydroxymethyl)-4H-pyran-4-one ...","""The chemical reactions and pathways involving...",[],[],[],[],NaN,[True],[0],[0],[False]


In [ ]:
bf = (bf.set_index('GO')
   .apply(lambda x: x.apply(pd.Series).stack())
   .reset_index()
   .drop('level_1', 1))

bf

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
0,GO:0000001,mitochondrion inheritance,biological_process,"['GO:0048308', 'GO:0048311']","['""mitochondrial inheritance"" EXACT []']","""The distribution of mitochondria, including t...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False
1,GO:0000002,mitochondrial genome maintenance,biological_process,['GO:0007005'],NaN,"""The maintenance of the structure and integrit...",NaN,NaN,NaN,NaN,NaN,False,80119.0,80119.0,PIF1
2,GO:0000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,84275.0,84275.0,SLC25A33
3,GO:0000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,92667.0,92667.0,MGME1
4,GO:0000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1890.0,1890.0,TYMP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73396,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0006040', 'GO:0006793', 'GO:0009225']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways involving...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False
73397,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009227', 'GO:0046348', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False
73398,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"['GO:0009226', 'GO:0046349', 'GO:2001313']","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...","""The chemical reactions and pathways resulting...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False
73399,GO:2001316,kojic acid metabolic process,biological_process,"['GO:0034308', 'GO:0042180', 'GO:0046483', 'GO...","['""5-hydroxy-2-(hydroxymethyl)-4H-pyran-4-one ...","""The chemical reactions and pathways involving...",NaN,NaN,NaN,NaN,NaN,True,0.0,0.0,False


In [ ]:
bf[bf.relationship.notnull()]

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment,notfound,_id,entrezgene,symbol
30,GO:0000018,regulation of DNA recombination,biological_process,GO:0051052,NaN,"""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0006310'],NaN,False,9984.0,9984.0,THOC1
36,GO:0000019,regulation of mitotic recombination,biological_process,GO:0000018,"['""regulation of recombination within rDNA rep...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0006312'],NaN,False,4361.0,4361.0,MRE11
38,GO:0000022,mitotic spindle elongation,biological_process,GO:0051231,"['""microtubule sliding involved in mitotic spi...","""The cell cycle process in which the distance ...",NaN,NaN,['GO:1905121'],"['part_of GO:0000070', 'part_of GO:0007052']",NaN,False,9493.0,9493.0,KIF23
46,GO:0000027,ribosomal large subunit assembly,biological_process,GO:0022618,"['""50S ribosomal subunit assembly"" NARROW [GOC...","""The aggregation, arrangement and bonding toge...",NaN,NaN,NaN,"['part_of GO:0042255', 'part_of GO:0042273']",NaN,False,140801.0,140801.0,RPL10L
56,GO:0000028,ribosomal small subunit assembly,biological_process,GO:0022618,"['""30S ribosomal subunit assembly"" NARROW [GOC...","""The aggregation, arrangement and bonding toge...",NaN,NaN,NaN,"['part_of GO:0042255', 'part_of GO:0042274']",NaN,False,6193.0,6193.0,RPS5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90486,GO:2001284,regulation of BMP secretion,biological_process,GO:0010646,"['""regulation of BMP protein secretion"" EXACT ...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0038055'],NaN,True,0.0,0.0,False
90489,GO:2001285,negative regulation of BMP secretion,biological_process,GO:0010648,"['""negative regulation of BMP protein secretio...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,['negatively_regulates GO:0038055'],NaN,True,0.0,0.0,False
90493,GO:2001286,regulation of caveolin-mediated endocytosis,biological_process,GO:0030100,"['""regulation of caveolae-dependent endocytosi...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0072584'],NaN,False,6714.0,6714.0,SRC
90494,GO:2001287,negative regulation of caveolin-mediated endoc...,biological_process,GO:0045806,"['""negative regulation of caveolae-dependent e...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,['negatively_regulates GO:0072584'],NaN,False,150696.0,150696.0,PROM2


In [ ]:
bf_map_cond = bf_map.groupby("query").agg({"notfound": lambda x: list(map(bool, x)),
                                    "_id": lambda x: list(map(int, x)),
                                   "entrezgene": lambda x: list(map(int, x)),
                                   "symbol": lambda x: list(map(str, x))})
bf_map_cond

,notfound,_id,entrezgene,symbol
query,,,,
GO:0000001,[True],[0],[0],[False]
GO:0000002,"[False, False, False, False, False, False, Fal...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[80119, 84275, 92667, 1890, 55186, 4358, 291, ...","[PIF1, SLC25A33, MGME1, TYMP, SLC25A36, MPV17,..."
GO:0000003,"[False, False, False, False]","[286826, 2797, 8510, 2796]","[286826, 2797, 8510, 2796]","[LIN9, GNRH2, MMP23B, GNRH1]"
GO:0000011,[True],[0],[0],[False]
GO:0000012,"[False, False, False, False, False, False, Fal...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[55775, 100133315, 1161, 2074, 200558, 54840, ...","[TDP1, XNDC1N, ERCC8, ERCC6, APLF, APTX, XRCC1..."
...,...,...,...,...
GO:2001313,[True],[0],[0],[False]
GO:2001314,[True],[0],[0],[False]
GO:2001315,[True],[0],[0],[False]


In [ ]:
bf_map_cond.loc["GO:0000002",]

notfound      [False, False, False, False, False, False, Fal...
_id           [80119, 84275, 92667, 1890, 55186, 4358, 291, ...
entrezgene    [80119, 84275, 92667, 1890, 55186, 4358, 291, ...
symbol        [PIF1, SLC25A33, MGME1, TYMP, SLC25A36, MPV17,...
Name: GO:0000002, dtype: object

In [ ]:
bf_map.groupby("query")

28503

In [ ]:
bf2[bf2.relationship.notnull()]

,GO,name,namespace,is_a,synonym,def,xref,subset,alt_id,relationship,comment
13,GO:0000018,regulation of DNA recombination,biological_process,['GO:0051052'],NaN,"""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0006310'],NaN
14,GO:0000019,regulation of mitotic recombination,biological_process,['GO:0000018'],"['""regulation of recombination within rDNA rep...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0006312'],NaN
15,GO:0000022,mitotic spindle elongation,biological_process,"['GO:0051231', 'GO:1903047']","['""microtubule sliding involved in mitotic spi...","""The cell cycle process in which the distance ...",NaN,NaN,['GO:1905121'],"['part_of GO:0000070', 'part_of GO:0007052']",NaN
20,GO:0000027,ribosomal large subunit assembly,biological_process,['GO:0022618'],"['""50S ribosomal subunit assembly"" NARROW [GOC...","""The aggregation, arrangement and bonding toge...",NaN,NaN,NaN,"['part_of GO:0042255', 'part_of GO:0042273']",NaN
21,GO:0000028,ribosomal small subunit assembly,biological_process,['GO:0022618'],"['""30S ribosomal subunit assembly"" NARROW [GOC...","""The aggregation, arrangement and bonding toge...",NaN,NaN,NaN,"['part_of GO:0042255', 'part_of GO:0042274']",NaN
...,...,...,...,...,...,...,...,...,...,...,...
43816,GO:2001284,regulation of BMP secretion,biological_process,"['GO:0010646', 'GO:0023051', 'GO:0050708']","['""regulation of BMP protein secretion"" EXACT ...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0038055'],NaN
43817,GO:2001285,negative regulation of BMP secretion,biological_process,"['GO:0010648', 'GO:0023057', 'GO:0050709', 'GO...","['""negative regulation of BMP protein secretio...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,['negatively_regulates GO:0038055'],NaN
43818,GO:2001286,regulation of caveolin-mediated endocytosis,biological_process,['GO:0030100'],"['""regulation of caveolae-dependent endocytosi...","""Any process that modulates the frequency, rat...",NaN,NaN,NaN,['regulates GO:0072584'],NaN
43819,GO:2001287,negative regulation of caveolin-mediated endoc...,biological_process,"['GO:0045806', 'GO:2001286']","['""negative regulation of caveolae-dependent e...","""Any process that stops, prevents or reduces t...",NaN,NaN,NaN,['negatively_regulates GO:0072584'],NaN


In [ ]:
bp[bp["query"] =="GO:0000002"]

,notfound,query,_id,symbol,_score,entrezgene
0,NaN,GO:0000002,80119,PIF1,16.833908,80119
0,NaN,GO:0000002,84275,SLC25A33,16.833908,84275
0,NaN,GO:0000002,92667,MGME1,16.833908,92667
0,NaN,GO:0000002,1890,TYMP,16.833908,1890
0,NaN,GO:0000002,55186,SLC25A36,16.833908,55186
0,NaN,GO:0000002,4358,MPV17,16.833908,4358
0,NaN,GO:0000002,291,SLC25A4,16.833908,291
0,NaN,GO:0000002,10000,AKT3,16.833908,10000
0,NaN,GO:0000002,4205,MEF2A,16.833908,4205
0,NaN,GO:0000002,4976,OPA1,16.833908,4976


In [ ]:
bf = nx.to_pandas_edgelist(GO, edge_key='type')
bf.iloc[5:20]

,source,target,type
5,GO:0000007,GO:0005385,is_a
6,GO:0000009,GO:0000030,is_a
7,GO:0000010,GO:0004659,is_a
8,GO:0000011,GO:0007033,is_a
9,GO:0000011,GO:0048308,is_a
10,GO:0000012,GO:0006281,is_a
11,GO:0000014,GO:0004520,is_a
12,GO:0000015,GO:1902494,is_a
13,GO:0000015,GO:0005829,part_of
14,GO:0000016,GO:0004553,is_a


In [ ]:
go = list(GO.nodes())[:5]

In [ ]:
#GO:0032736 -AdjacencyView({'GO:0001819': {'is_a': {}}, 'GO:0032656': {'is_a': {}}, 'GO:0032616': {'positively_regulates': {}}})
GO["GO:0032736"]

AdjacencyView({'GO:0001819': {'is_a': {}}, 'GO:0032656': {'is_a': {}}, 'GO:0032616': {'positively_regulates': {}}})

In [ ]:
GO[('GO:0000001', 'GO:0048308')]

KeyError: ignored

In [ ]:
try:
    import mygene
except ModuleNotFoundError as err:
    !pip install mygene
    import mygene
    # Error handling
    print(err)
 
mg = mygene.MyGeneInfo()
out = mg.querymany(go, scopes='GO', fields='entrezgene,symbol', species='human', returnall=True)




No module named 'mygene'
querying 1-5...done.
Finished.
2 input query terms found dup hits:
	[('GO:0000002', 10), ('GO:0000003', 4)]
3 input query terms found no hit:
	['GO:0000001', 'GO:0000006', 'GO:0000007']


In [ ]:
out["out"]

[{'notfound': True, 'query': 'GO:0000001'},
 {'_id': '80119',
  '_score': 16.811384,
  'entrezgene': '80119',
  'query': 'GO:0000002',
  'symbol': 'PIF1'},
 {'_id': '84275',
  '_score': 16.811384,
  'entrezgene': '84275',
  'query': 'GO:0000002',
  'symbol': 'SLC25A33'},
 {'_id': '92667',
  '_score': 16.811384,
  'entrezgene': '92667',
  'query': 'GO:0000002',
  'symbol': 'MGME1'},
 {'_id': '1890',
  '_score': 16.811384,
  'entrezgene': '1890',
  'query': 'GO:0000002',
  'symbol': 'TYMP'},
 {'_id': '55186',
  '_score': 16.811384,
  'entrezgene': '55186',
  'query': 'GO:0000002',
  'symbol': 'SLC25A36'},
 {'_id': '4358',
  '_score': 16.811384,
  'entrezgene': '4358',
  'query': 'GO:0000002',
  'symbol': 'MPV17'},
 {'_id': '291',
  '_score': 16.811384,
  'entrezgene': '291',
  'query': 'GO:0000002',
  'symbol': 'SLC25A4'},
 {'_id': '10000',
  '_score': 16.811384,
  'entrezgene': '10000',
  'query': 'GO:0000002',
  'symbol': 'AKT3'},
 {'_id': '4976',
  '_score': 16.811384,
  'entrezgene':

In [ ]:
import goatools 
GO.graph.keys()

dict_keys(['typedefs', 'instances', 'format-version', 'data-version', 'subsetdef', 'synonymtypedef', 'default-namespace', 'ontology', 'name'])

In [ ]:
GO.graph["typedefs"]

[{'id': 'negatively_regulates',
  'is_a': ['regulates'],
  'name': 'negatively regulates',
  'namespace': 'external',
  'xref': ['RO:0002212']},
 {'id': 'part_of',
  'is_transitive': 'true',
  'name': 'part of',
  'namespace': 'external',
  'xref': ['BFO:0000050']},
 {'holds_over_chain': ['negatively_regulates negatively_regulates'],
  'id': 'positively_regulates',
  'is_a': ['regulates'],
  'name': 'positively regulates',
  'namespace': 'external',
  'xref': ['RO:0002213']},
 {'id': 'regulates',
  'is_transitive': 'true',
  'name': 'regulates',
  'namespace': 'external',
  'xref': ['RO:0002211']},
 {'id': 'term_tracker_item',
  'is_class_level': 'true',
  'is_metadata_tag': 'true',
  'name': 'term tracker item',
  'namespace': 'external',
  'xref': ['IAO:0000233']}]

In [ ]:
# check the number of edges
pd.Series([i[2] for i in GO.edges]).value_counts()

is_a                    70881
part_of                  7096
regulates                3208
negatively_regulates     2761
positively_regulates     2751
dtype: int64

In [ ]:
df

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,blackBoxWarning,drugType,ActionType_drug__filter,tradeNames,hasBeenWithdrawn,isApproved,withdrawnNotice,yearOfFirstApproval,maximumClinicalTrialPhase,DrugTargetType_drug__filter
0,381,375,ARF5,ARF1,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,381,9727,ARF5,RAB11FIP3,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,381,2108,ARF5,ETFA,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,381,23647,ARF5,ARFIP2,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4074,26088,M6PR,GGA1,disease_related_protein,disease_related_protein,protein,protein,red,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205575,CHEMBL1631540,6532,GSK163090,SLC6A4,drug,drug_target,drug,protein,green,s,green,o,NaN,0.0,Small molecule,ANTAGONIST,[],0.0,0.0,NaN,NaN,2.0,single protein
205576,CHEMBL4297187,2261,DERAZANTINIB,FGFR3,drug,drug_target,drug,protein,green,s,green,o,NaN,0.0,Small molecule,INHIBITOR,[],0.0,0.0,NaN,NaN,2.0,protein family
205577,CHEMBL4297187,2260,DERAZANTINIB,FGFR1,drug,drug_target,drug,protein,green,s,green,o,NaN,0.0,Small molecule,INHIBITOR,[],0.0,0.0,NaN,NaN,2.0,protein family
205578,CHEMBL4297187,2264,DERAZANTINIB,FGFR4,drug,drug_target,drug,protein,green,s,green,o,NaN,0.0,Small molecule,INHIBITOR,[],0.0,0.0,NaN,NaN,2.0,protein family


In [ ]:
pd.append(df.loc[:,["node1", "node1_type"]], df.loc[:,["node2", "node2_type"]])

AttributeError: ignored

In [ ]:
df.loc[:,["node2", "node2_type"]].rename(columns={"node2": "node", "node2_type": "node_type"}).append(df.loc[:,["node1", "node1_type"]].rename(columns={"node1": "node", "node1_type": "node_type"})).drop_duplicates(subset=['node']).value_counts("node_type")

node_type
protein                    15115
disease                     9347
drug                        4369
disease_related_protein     2926
drug_target                 1985
dtype: int64